In [1]:
from re import findall
from retrying import retry
from sqlite3 import connect
from bs4 import BeautifulSoup
from pydantic import BaseModel
from requests import Session, Response

In [2]:
URL_BASE = 'https://www.vintepila.com.br/'

In [3]:
data_login = {
    "log": "",
    "miniorange_login_nonce": "",
    "redirect_to": "https://www.vintepila.com.br/minha-conta/",
    "pwd": "",
    "miniorange_login_nonce": "",
}
session = Session()

In [4]:
def get_request(url: str, cookies: dict = None, **args: dict) -> Response:
    return session.get(url, cookies=cookies, **args)

def post_request(url: str, cookies: dict = None, data: dict = None, **args: dict) -> Response:
    return session.post(url, cookies=cookies, data=data, **args)

In [5]:
def login_page():
    return session.post(URL_BASE + 'login/?wpe-login=btcuser', data=data_login)

In [6]:
def get_pages() -> int:
    response = get_request(URL_BASE+'trabalhos-freelance')
    soup = BeautifulSoup(response.text, 'html.parser')
    pages = soup.find_all('a', class_='page-numbers')[2].text
    return int(pages)


In [7]:
def extract_data_link(url: str, page: int) -> list:
    response = get_request(url + 'trabalhos-freelance?page={}'.format(page))
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('div', class_='ui segment')
    return [link.find('a')['href'] for link in links if link.find('a')]


In [14]:
extract_name_candidate = lambda soup: soup.find('div', class_='header').text

def get_content_page(url: str) -> dict:
    response = get_request(url)
    if response.status_code != 200:
        return {}
    response.status_code
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find_all('div', class_='header')[1].text
    description = soup.find('div', class_='description').text
    candidate = soup.find('div', class_='extra').find_all('span')[2].text
    data_published = soup.find('div', class_='extra').find_all('span')[0].text
    price = soup.find_all('span', class_='ui green header')[0].text.split(' ')[-1][:-1]
    project = soup.find_all('div', class_='ui fluid container')[0].find('span').text.split(' ')[-1]
    delivery = soup.find('span', {'data-content':'Prazo máximo estipulado pelo cliente para a entrega do serviço'}).text
    candiatures_name = [extract_name_candidate(candidate) for candidate in soup.find_all('div', class_='ui segment')]
    return {'title': title,
            'candiates_name': candiatures_name,
            'description': description,
            'price': price,
            'data_published': data_published,
            'candidate': candidate,
            'delivery': delivery,
            'project': project,
            'link': url
    }


In [11]:
def caditate_job(url: str, price: str, delivery: str, associate_project_id: str) ->bool:
    url += 'wp-admin/admin-ajax.php'
    payload = {
        "price": price,
        "delivery": delivery,
        "action": "custom_project_offer_submit",
        "formrandomid": "1",
        "user": "652909",
        "associate_project_id": associate_project_id,
    }
    response = post_request(url, data=payload)
    if response.status_code != 200:
        print(f'Error found with {payload} and {url}')
        return False
    print(f'Candidated with success in {url}, {payload}')
    return True
    

In [11]:
login_page()
pages = get_pages()
for page in range(1, pages+1):
    print(f'change page: {page}')
    links = extract_data_link(URL_BASE, page)
    for link in links:
        print(get_content_page(link))

change page: 1
{'title': 'Preciso de editor de pagina de venda e checkout para um projeto ja em andamento', 'description': '\n Preciso de um editor de pagina de venda com vsl de alta conversão se for copy tambem será um adicional e tambem que refaça a pagina de checkout do produto, é um projeto grande dependendo do resultado vou precisar da pessoa na minha equipe de desenvolvimento\n\n', 'price': '200,00', 'data_published': '23 horas', 'candidate': '13', 'delivery': '2 DIAS', 'project': '4193543', 'link': 'https://www.vintepila.com.br/trabalhos-freelance/preciso-de-editor-de-pagina-de-venda-e-checkout-para-um-projeto-ja-em-andamento/'}
{'title': 'Preciso de Homem acima de 50 anos para fazer uma propaganda sacando 900 na boca do caixa', 'description': '\n – Preciso de um homem acima de 50 anos para fazer uma propaganda sacando 900 na boca do caixa\n– Irei enviar o script, vídeo com menos de 30 segundos\n– Nicho renda extra\n\n', 'price': '50,00', 'data_published': '14 junho às 09:49', '

In [16]:
login_page()
pages = get_pages()
for page in range(1, pages+1):
    print(f'change page: {page}')
    links = extract_data_link(URL_BASE, page)
    for link in links:
        response = get_content_page(link)
        if 'Gregorio h.' not in response['candiates_name']:
            caditate_job(URL_BASE, response['price'], response['delivery'], response['project'])
            print(response)

change page: 1
Candidated with success in https://www.vintepila.com.br/wp-admin/admin-ajax.php, {'price': '50,00', 'delivery': '2 DIAS', 'action': 'custom_project_offer_submit', 'formrandomid': '1', 'user': '652909', 'associate_project_id': '4205046'}
{'title': 'Depoimento feminino para anúncio em Redes Sociais', 'candiates_name': ['Luciene P.', 'Júlia G.', 'Jeana D.', 'Elaine R.', 'Hiasminy M.'], 'description': '\n Preciso de um depoimento feminino para ser usado em um anúncio em redes sociais, deve atuar como uma empresária do ramo da beleza, como uma cabeleireira, falando sobre como o meu curso ajudou a aprender a precificar os serviços corretamente.\n\n', 'price': '50,00', 'data_published': '3 horas', 'candidate': '5', 'delivery': '2 DIAS', 'project': '4205046', 'link': 'https://www.vintepila.com.br/trabalhos-freelance/depoimento-feminino-para-anuncio-em-redes-sociais/'}
Candidated with success in https://www.vintepila.com.br/wp-admin/admin-ajax.php, {'price': '100,00', 'delivery':